In [47]:
# Author: Yijun Xiao <ryjxiao@nyu.edu>

## 0. Data Loader
Created to help fetch data from MTA website

In [219]:
from __future__ import division, print_function
import pandas as pd
from pandas import datetime, Timedelta
import urllib2
import os
import cPickle as pickle

In [341]:
class TurnstileDataLoader:
    """Automatically load data from MTA website"""
    
    def __init__(self):
        # format of links to the txt files
        self.url_base = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{0}.txt"
        # first day of data
        self.begining_of_time = datetime(2010, 5, 1)
        # date when format of data changed
        self.new_era = datetime(2014, 10, 18)
        self.today = datetime.today()
        
        # prepare station df for old format data
        self.data_dir = "data/"
        station_df_path = os.path.join(self.data_dir, "station.pkl")
        if os.path.isfile(station_df_path):
            with open(station_df_path) as f:
                self.station_df = pickle.load(f)
        else:
            self.station_df = pd.read_excel("http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls")
            self.station_df.columns = ["UNIT", "C/A", "STATION", "LINENAME", "DIVISION"]
            # save to data directory
            if not os.path.exists(self.data_dir):
                os.makedirs(self.data_dir)
            with open(station_df_path, "wb") as f:
                pickle.dump(self.station_df, f)
        
    def _next_saturday(self, date):
        """Find the nearest saturday after input date when data updated"""
        weekday = date.weekday()
        delta = Timedelta(7 - (weekday + 2) % 7, unit='d')
        return date + delta
    
    def _find_files(self, start_date, end_date):
        """Find list of files covering specified starting date and 
        end date."""
        # some assertions
        assert(start_date < end_date)
        assert(start_date >= self.begining_of_time)
        # assert data is available on mta website
        assert(self._next_saturday(end_date) <= self.today)
        
        files = []
        start_saturday = self._next_saturday(start_date)
        end_saturday = self._next_saturday(end_date)
        while start_saturday <= end_saturday:
            datestr = start_saturday.strftime("%y%m%d")
            files.append(self.url_base.format(datestr))
            start_saturday += Timedelta("7 days")
        return files
    
    def _load_old(self, txtfileurl):
        """Load old format txt file which needs quite some reformating"""
        records = []
        txtfile = urllib2.urlopen(txtfileurl)
        for line in txtfile:
            row = line.strip().split(",")
            if len(row) < 8:
                continue
            ca, unit, scp = row[:3]
            i = 3
            while i < len(row):
                date, time, desc, entries, exits = row[i:i+5]
                date_time = datetime.strptime(date + " " + time, "%m-%d-%y %H:%M:%S")
                record = dict(DATE_TIME=date_time, UNIT=unit, SCP=scp,
                              DESC=desc, ENTRIES=int(entries), EXITS=int(exits))
                record["C/A"] = ca
                records.append(record)
                i += 5
        old_df = pd.DataFrame.from_records(records)
        return pd.merge(old_df, self.station_df, how="left").set_index(["DATE_TIME"])
        
    def load(self, txtfileurl):
        """Load txt file specified by a url as a DataFrame"""
        datestr = txtfileurl[-10:-4]
        # check if we have the data saved already
        filepath = os.path.join(self.data_dir, datestr + ".pkl")
        if os.path.isfile(filepath):
            with open(filepath) as f:
                result = pickle.load(f)
        else:
            # detect whether this file is of the newer, cleaner format
            is_new = datetime.strptime(datestr, "%y%m%d") >= self.new_era
            if is_new:
                result = pd.read_csv(txtfileurl, parse_dates=[[6,7]], index_col=["DATE_TIME"]).sort_index()
                result.columns = ["C/A", "UNIT", "SCP", "STATION",
                                  "LINENAME", "DIVISION", "DESC", "ENTRIES", "EXITS"]
            else:
                result = self._load_old(txtfileurl).sort_index()
                
            result["TURNSTILE_ID"] = result[["C/A", "UNIT", "SCP"]].apply(lambda x: " ".join(x), axis=1)
            result.drop(["C/A", "UNIT", "SCP"], axis=1, inplace=True)
            with open(filepath, "wb") as f:
                pickle.dump(result, f)
                
        return result
            
    def retrieve(self, start_date, end_date):
        """Retrieve data given starting date and end date.
        Note that starting date is included while end date is not.
        """
        files = self._find_files(start_date, end_date)
        frames = []
        for fileurl in files:
            frames.append(self.load(fileurl))
        result = pd.concat(frames)
        # need to include the first entry of end date to calculate count
        # the latest first entry on any day is 3am
        end_time = end_date.replace(hour=3, minute=0, second=0, microsecond=0)
        return result[start_date:end_time]

## 1. Total number of entries & exits
across the subway system for Aug. 1st, 2013

In [342]:
data_loader = TurnstileDataLoader()

In [343]:
start_date = datetime(2013, 8, 1)
end_date = datetime(2013, 8, 2)
df = data_loader.retrieve(start_date, end_date)

In [329]:
# read "odometers" at the beginning and the end
start_count = df["2013-08-01 00:00:00"][["TURNSTILE_ID", "STATION", "ENTRIES", "EXITS"]]
end_count = df["2013-08-02 00:00:00"][["TURNSTILE_ID", "STATION", "ENTRIES", "EXITS"]]

In [330]:
# We have different number of turnstiles recorded at midnight
# Aug. 1st and Aug. 2nd. Thus it's not okay to simply sum up
# all entries and exits at both time points and do subtraction
print(len(start_count))
print(len(end_count))

2360
2365


In [331]:
# check the set of turnstiles in both time points
turnstile_setA = set()
turnstile_setB = set()
for _, row in start_count.iterrows():
    turnstile_setA.add(row["TURNSTILE_ID"])
for _, row in end_count.iterrows():
    turnstile_setB.add(row["TURNSTILE_ID"])

In [332]:
# calculate the intersection
set_both = turnstile_setA.intersection(turnstile_setB)

In [333]:
# compute the mask for each df, only keep turnstiles in intersection
maskA = start_count.apply(lambda x: x["TURNSTILE_ID"] in set_both, axis=1)
maskB = end_count.apply(lambda x: x["TURNSTILE_ID"] in set_both, axis=1)

In [334]:
# we can see here something is wrong with our data
# as total odometer reading on Aug. 2nd is smaller
odometerA = start_count[maskA][["ENTRIES", "EXITS"]].sum()
odometerB = end_count[maskB][["ENTRIES", "EXITS"]].sum()

In [362]:
# some odometers have been reset during the time frame
bad_entries_diff = 0
bad_exits_diff = 0
bad_turnstiles = set()
for tid in set_both:
    rowA = start_count[start_count["TURNSTILE_ID"] == tid]
    rowB = end_count[end_count["TURNSTILE_ID"] == tid]
    if (rowA.ENTRIES.sum() > rowB.ENTRIES.sum()) | (rowA.EXITS.sum() > rowB.EXITS.sum()):
        bad_entries_diff += rowA.ENTRIES.sum() - rowB.ENTRIES.sum()
        bad_exits_diff += rowA.EXITS.sum() - rowB.EXITS.sum()
        bad_turnstiles.add(tid)

In [382]:
# we ignore resetted odometers here. This count should be smaller than
# real count
total_traffic = (odometerB - odometerA).sum() + bad_entries_diff + bad_exits_diff
print("There are a total of {0:,} entries and exits on Aug. 1st, 2013".format(total_traffic))

There are a total of 5,704,970 entries and exits on Aug. 1st, 2013


## 2. Busiest station and turnstile
on Aug. 1st, 2013

In [363]:
# redefine masks to further filter out resetted turnstiles
valid_turnstiles = set_both - bad_turnstiles
maskA = start_count.apply(lambda x: x["TURNSTILE_ID"] in valid_turnstiles, axis=1)
maskB = end_count.apply(lambda x: x["TURNSTILE_ID"] in valid_turnstiles, axis=1)

In [367]:
# sum of odometer readings grouped by station
station_countA = start_count[maskA].groupby("STATION").sum()
station_countB = end_count[maskB].groupby("STATION").sum()

In [370]:
# total traffic of both entries and exits in each station
station_traffic = (station_countB - station_countA).sum(1)

In [373]:
busiest_station = station_traffic.argmax()
largest_station_traffic = station_traffic.max()

In [381]:
print("The busiest station on Aug. 1st, 2013 is {0}".format(busiest_station))
print("Total entries & exits: {0:,}".format(largest_station_traffic))

The busiest station on Aug. 1st is 34 ST-HERALD SQ
Total entries & exits: 237,077


In [377]:
turnstile_countA = start_count[maskA].groupby("TURNSTILE_ID").sum()
turnstile_countB = end_count[maskB].groupby("TURNSTILE_ID").sum()
turnstile_traffic = (turnstile_countB - turnstile_countA).sum(1)

In [378]:
busiest_turnstile = turnstile_traffic.argmax()
largest_turnstile_traffic = turnstile_traffic.max()

In [386]:
# check which station this turnstile is located
at_station = start_count[start_count["TURNSTILE_ID"]==busiest_turnstile]["STATION"].values[0]

In [416]:
print("The busiest turnstile on Aug. 1st, 2013 is {0}".format(busiest_turnstile))
print("Located at {0}".format(at_station))
print("Total entries & exits: {0:,}".format(largest_turnstile_traffic))

The busiest turnstile on Aug. 1st is N063A R011 00-00-00
Located at 42 ST-PA BUS TE
Total entries & exits: 11,845


## 3. Busiest and least-busy stations
in July, 2013

In [422]:
# load data
start_date = datetime(2013, 7, 1)
end_date = datetime(2013, 8, 1)
df2 = data_loader.retrieve(start_date, end_date)

In [407]:
# automate the masking process
def get_valid_readings(start_df, end_df):
    setA = set()
    setB = set()
    for _, row in start_df.iterrows():
        setA.add(row["TURNSTILE_ID"])
    for _, row in end_df.iterrows():
        setB.add(row["TURNSTILE_ID"])
    valid_set = setA.intersection(setB)
    bad_set = set()
    for tid in valid_set:
        rowA = start_df[start_df["TURNSTILE_ID"] == tid]
        rowB = end_df[end_df["TURNSTILE_ID"] == tid]
        if (rowA.ENTRIES.sum() > rowB.ENTRIES.sum()) | (rowA.EXITS.sum() > rowB.EXITS.sum()):
            bad_set.add(tid)
    valid_set -= bad_set
    maskA = start_df.apply(lambda x: x["TURNSTILE_ID"] in valid_set, axis=1)
    maskB = end_df.apply(lambda x: x["TURNSTILE_ID"] in valid_set, axis=1)
    return maskA, maskB

In [405]:
start_df = df2["2013-07-01 00:00:00"][["TURNSTILE_ID", "STATION", "ENTRIES", "EXITS"]]
end_df = df2["2013-08-01 00:00:00"][["TURNSTILE_ID", "STATION", "ENTRIES", "EXITS"]]

In [408]:
maskA, maskB = get_valid_readings(start_df, end_df)

In [411]:
# same analysis as in part 2
station_countA = start_df[maskA].groupby("STATION").sum()
station_countB = end_df[maskB].groupby("STATION").sum()
station_traffic = (station_countB - station_countA).sum(1)

In [417]:
busiest_station = station_traffic.argmax()
largest_station_traffic = station_traffic.max()
print("The busiest station in July, 2013 is {0}".format(busiest_station))
print("Total entries & exits: {0:,}".format(largest_station_traffic))

The busiest station in July, 2013 is 34 ST-HERALD SQ
Total entries & exits: 6,198,212


In [419]:
least_station = station_traffic.argmin()
smallest_station_traffic = station_traffic.min()
print("The least-busy station in July, 2013 is {0}".format(least_station))
print("Total entries & exits: {0:,}".format(smallest_station_traffic))

The least-busy station in July, 2013 is AQUEDUCT TRACK
Total entries & exits: 269


## 4. Station with highest average number of entries 
between midnight & 4am on Fridays in July 2013

In [450]:
from datetime import time

In [457]:
# get all data in the time frame of interest
m = df2.index.map(lambda x: (x.weekday() == 5) and (x.time() <= time(4, 0, 0)))
df3 = df2[m]

In [458]:
df3

,DESC,ENTRIES,EXITS,STATION,LINENAME,DIVISION,TURNSTILE_ID
DATE_TIME,,,,,,,
2013-07-06 00:00:00,REGULAR,4182206,1435880,LEXINGTON AVE,456NQR,BMT,A002 R051 02-00-00
2013-07-06 00:00:00,REGULAR,0,812,33 ST/RAWSON ST,7,IRT,R517 R291 01-05-01
2013-07-06 00:00:00,REGULAR,3030860,3777678,METROPOLITAN AV,M,BMT,K026 R100 00-00-02
2013-07-06 00:00:00,REGULAR,4698277,3585098,METROPOLITAN AV,M,BMT,K026 R100 00-00-01
2013-07-06 00:00:00,REGULAR,117703141,149177,33 ST/RAWSON ST,7,IRT,R517 R291 01-06-00
2013-07-06 00:00:00,REGULAR,1780732,1302974,METROPOLITAN AV,M,BMT,K026 R100 00-00-00
2013-07-06 00:00:00,REGULAR,3546509,4920296,33 ST/RAWSON ST,7,IRT,R517 R291 01-06-01
2013-07-06 00:00:00,REGULAR,4647262,6403512,METROPOLITAN AV,M,BMT,K026 R100 00-00-03
2013-07-06 00:00:00,REGULAR,3308280,8752994,40 ST-LOWERY ST,7,IRT,R518 R261 00-00-00
